In [1]:
import pandas as pd
import numpy as np

import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

from sentence_transformers import SentenceTransformer
sentences_1 = ["样例数据-1", "样例数据-2"]
model = SentenceTransformer('BAAI/bge-small-zh-v1.5')
embeddings_1 = model.encode(sentences_1, normalize_embeddings=True, show_progress_bar=True)

/home/lyz/anaconda3/envs/py311/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/lyz/anaconda3/envs/py311/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
import time
import jwt
import requests

# 实际KEY，过期时间
def generate_token(apikey: str, exp_seconds: int):
    try:
        id, secret = apikey.split(".")
    except Exception as e:
        raise Exception("invalid apikey", e)

    payload = {
        "api_key": id,
        "exp": int(round(time.time() * 1000)) + exp_seconds * 1000,
        "timestamp": int(round(time.time() * 1000)),
    }
    return jwt.encode(
        payload,
        secret,
        algorithm="HS256",
        headers={"alg": "HS256", "sign_type": "SIGN"},
    )

def glm4air(prompt):
    url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
    headers = {
      'Content-Type': 'application/json',
      'Authorization': generate_token("填入你的 key", 1000)
    }
    
    data = {
        "model": "glm-4-air",
        "messages": [{"role": "user", "content":  prompt }]
    }
    
    response = requests.post(url, headers=headers, json=data, timeout=200)
    return response.json()['choices'][0]['message']['content']

glm4air('早上好')

'早上好🌞！新的一天开始了，祝您心情愉快，工作顺利！有什么可以帮助您的吗？'

In [18]:
corpus = open('corpus.txt').readlines()
query = pd.read_csv('test_question.csv')['question'].values

In [19]:
query_feat = model.encode(query, normalize_embeddings=True, show_progress_bar=True)
corpus_feat = model.encode(corpus, normalize_embeddings=True, show_progress_bar=True)

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

In [26]:
from tqdm import tqdm
test_answer = []
for q, qtext in tqdm(zip(query_feat, query)):
    top1_content = corpus[np.dot(q, corpus_feat.T).argsort()[-1]]
    prompt_text = f'''请结合下面的资料，回答给定的问题：

提问：{qtext}

相关资料：{top1_content}
'''

    try:
        answer = glm4air(prompt_text)
        answer = answer.replace('\n', '')
        test_answer.append(answer)
    except:
        test_answer.append('无法回答。')

332it [22:10,  4.01s/it]


In [28]:
test_answer = pd.DataFrame(test_answer)
test_answer.columns = ['answer']

In [29]:
test_answer.to_csv('submit.csv', index=None)